# DATA SCIENCE UTS Praktikum
TUGAS

*   Dari dataset titanic, training data dengan 4 metode ML (KNN, Naive Bayes, Decision Tree, dan SVM)


*   Lakukan preprocessing sebelum melakukan training

*   Gunakan 5 fold cross validation dan splitting training dan testing dengan perbandingan 70%, 30%

*   Gunakan Measurement dengan metrix Akurasi, ROC-curve, precision, recall, Sensitivity, specificity, dan F1-Measurement

In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer

Memuat dan membaca dataset titanic kemudian ditampilkan 5 data teratas.

In [ ]:
dataset = pd.read_csv('titanic.csv', na_values='?')
ds = dataset.copy()
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Menampilkan jumlah missing value setiap atribut.

In [ ]:
ds.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Melihat tipe data dari masing - masing kolom dataset


In [ ]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Memisah antara data training dan data target dengan X sebagai data training dan y sebagai target

In [ ]:
X, y = ds.drop(['Survived'], axis =1), ds.Survived

*   Preprocessing pada setiap atribut dengan mengubah dari data kategorical kedalam data numeric dan menangani missing value.

*   Pada atribut sex, data dipetakan menjadi 1 untuk male dan 0 untuk female. 

*   Pada age dan embarked terdapat missing value, maka pada age, missing data akan diisi dengan rata – rata atribut pada kelas yang sama
*   Pada atribut embarked karena data teermasuk kategorical, maka data yang hilaing diisi dengan nilai yang paling sering muncul pada atribut tersebut (modus) setelah itu atribut diubah menjadi data numeric. 





In [ ]:
le = preprocessing.LabelEncoder()
X['Sex'] = ds['Sex'].map({'male':1, 'female':0})
X['Age'] = ds.groupby('Survived')['Age'].transform(lambda x: x.fillna(round(x.mean(), 0)))
X['Ticket'] = le.fit_transform(X.Ticket)

X['Embarked'].fillna(ds.groupby('Survived')['Embarked'].transform(lambda x: pd.Series.mode(x)[0]), inplace = True)
X['Embarked'] = le.fit_transform(X.Embarked)

X.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",1,22.0,1,0,523,7.2500,NaN,2
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,596,71.2833,C85,0
2,3,3,"Heikkinen, Miss. Laina",0,26.0,0,0,669,7.9250,NaN,2
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,49,53.1000,C123,2
4,5,3,"Allen, Mr. William Henry",1,35.0,0,0,472,8.0500,NaN,2


Pada atribut name, karena tidak terlalu berpengaruh pada target maka akan dipisah berdasarkan gelar mereka.

In [ ]:
titles = set()
for name in X['Name']:
    titles.add(name.split(',')[1].split('.')[0].strip())

print(titles)
Title_Dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

{'Ms', 'Mrs', 'Capt', 'Mme', 'Rev', 'Miss', 'Lady', 'Master', 'the Countess', 'Col', 'Mr', 'Major', 'Jonkheer', 'Dr', 'Don', 'Mlle', 'Sir'}


Menggunakan teknik one hot encoding untuk membentuk atribut baru sesuai dengan gelar dan menghapus atribut nama dalam dataset

In [ ]:
X['Title'] = X['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
X['Title'] = X.Title.map(Title_Dictionary)

titles_dummies = pd.get_dummies(X['Title'], prefix='Title')
X = pd.concat([X, titles_dummies], axis=1)
X = X.drop(columns=['Name', 'Title'])
X.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty
0,1,3,1,22.0,1,0,523,7.2500,NaN,2,0,0,1,0,0,0
1,2,1,0,38.0,1,0,596,71.2833,C85,0,0,0,0,1,0,0
2,3,3,0,26.0,0,0,669,7.9250,NaN,2,0,1,0,0,0,0
3,4,1,0,35.0,1,0,49,53.1000,C123,2,0,0,0,1,0,0
4,5,3,1,35.0,0,0,472,8.0500,NaN,2,0,0,1,0,0,0


Atribut kabin memiliki missing value yang cukup banyak
hal yang dilakukan adalah memetakan data dan membentuk atribut baru sesuai dengan karakter pertama dari nama kabin dan missing value diletakkan pada atribut Cabin_U yang berarti "Unknown"

In [ ]:
X.Cabin.fillna('U', inplace=True)
    
    # mapping each Cabin value with the cabin letter
X['Cabin'] = X['Cabin'].map(lambda c: c[0])
    
    # dummy encoding ...
cabin_dummies = pd.get_dummies(X['Cabin'], prefix='Cabin')    
X = pd.concat([X, cabin_dummies], axis=1)
X = X.drop(columns=['Cabin'])
X.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U
0,1,3,1,22.0,1,0,523,7.2500,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,2,1,0,38.0,1,0,596,71.2833,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
2,3,3,0,26.0,0,0,669,7.9250,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,4,1,0,35.0,1,0,49,53.1000,2,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
4,5,3,1,35.0,0,0,472,8.0500,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


Melakukan normalisasi dengan metode Z score

In [ ]:
#NORMALISASI
Xzscore = scaler.fit_transform(X)
Xzscore

array([[-1.73010796,  0.82737724,  0.73769513, ..., -0.06715343,
        -0.03352008,  0.54492498],
       [-1.72622007, -1.56610693, -1.35557354, ..., -0.06715343,
        -0.03352008, -1.835115  ],
       [-1.72233219,  0.82737724, -1.35557354, ..., -0.06715343,
        -0.03352008,  0.54492498],
       ...,
       [ 1.72233219,  0.82737724, -1.35557354, ..., -0.06715343,
        -0.03352008,  0.54492498],
       [ 1.72622007, -1.56610693,  0.73769513, ..., -0.06715343,
        -0.03352008, -1.835115  ],
       [ 1.73010796,  0.82737724,  0.73769513, ..., -0.06715343,
        -0.03352008,  0.54492498]])

In [ ]:
specificity = make_scorer(recall_score, pos_label=0)
kf = KFold(n_splits=5)
X_train, X_test, y_train, y_test = train_test_split(Xzscore, y, test_size=0.30, random_state=0)

Melakukan KNN dengan 5 fold validation

In [ ]:
#KNN 5-FOLD VALIDATION
knn = KNeighborsClassifier(n_neighbors=5)
knn_scores = cross_val_score(knn, Xzscore, y, scoring = 'accuracy', cv = kf) 
knn_scores_ROC = cross_val_score(knn, Xzscore, y, scoring = 'roc_auc', cv=kf)
knn_scores_precision = cross_val_score(knn, Xzscore, y, scoring = 'precision', cv=kf)
knn_scores_recall = cross_val_score(knn, Xzscore, y, scoring = 'recall', cv=kf)
knn_scores_specificity = cross_val_score(knn, Xzscore, y, scoring = specificity, cv=kf)
knn_scores_f1 = cross_val_score(knn, Xzscore, y, scoring = 'f1', cv=kf)

print("Accuracy: ", np.mean(knn_scores))
print("AUC: ", knn_scores_ROC.mean(), "", knn_scores_ROC.std())
print("Precision: ", np.mean(knn_scores_precision))
print("Recall: ", np.mean(knn_scores_recall))
print("Sensitivity: ", np.mean(knn_scores_recall))
print("Specificity: ", np.mean(knn_scores_specificity))
print("f1-measure: ", np.mean(knn_scores_f1))

Accuracy:  0.8136965664427847
AUC:  0.8553529969489846  0.033363704134740345
Precision:  0.7991535817434813
Recall:  0.6944971074755222
Sensitivity:  0.6944971074755222
Specificity:  0.8915865848161835
f1-measure:  0.738936434108527


Melakukan KNN dengan train test split rasio 7:3.

In [ ]:
#KNN Splitting training and testing (70%, 30%)

knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
knn_scores2 = metrics.accuracy_score(y_test, y_pred)

tn, fp, fn, tp = matrix = confusion_matrix(y_test, y_pred).ravel()
knn_roc = roc_auc_score(y_test, y_pred)
knn_precision=tp/(tp+fp)
knn_recall=tp/(tp+fn)
knn_nvp=tn/(tn+fn)
knn_specificity=tn/(tn+fp)
knn_f1measure=2*(knn_precision*knn_recall)/(knn_precision+knn_recall)

print("Accuracy: ", knn_scores2)
print("ROC: ", knn_roc)
print("Precision: ", knn_precision)
print("Recall: ", knn_recall)
print("Sensitivity: ", knn_recall)
print("Specificity: ", knn_specificity)
print("F1-Measure: ", knn_f1measure)

Accuracy:  0.8059701492537313
ROC:  0.7946428571428572
Precision:  0.7352941176470589
Recall:  0.75
Sensitivity:  0.75
Specificity:  0.8392857142857143
F1-Measure:  0.7425742574257426


Naive Bayes 5-FOLD VALIDATION

In [ ]:
#Naive Bayes 5-FOLD VALIDATION
NB = GaussianNB()
NB_scores = cross_val_score(NB, Xzscore, y, scoring = 'accuracy', cv=kf)
NB_scores_ROC = cross_val_score(NB, Xzscore, y, scoring = 'roc_auc', cv=kf)
NB_scores_precision = cross_val_score(NB, Xzscore, y, scoring = 'precision', cv=kf)
NB_scores_recall = cross_val_score(NB, Xzscore, y, scoring = 'recall', cv=kf)
NB_scores_specificity = cross_val_score(NB, Xzscore, y, scoring = specificity, cv=kf)
NB_scores_f1 = cross_val_score(NB, Xzscore, y, scoring = 'f1', cv=kf)

print("Accuracy: ", np.mean(NB_scores))
print("AUC: ", NB_scores_ROC.mean(), "", NB_scores_ROC.std())
print("Precision: ", np.mean(NB_scores_precision))
print("Recall: ", np.mean(NB_scores_recall))
print("Sensitivity: ", np.mean(NB_scores_recall))
print("Specificity: ", np.mean(NB_scores_specificity))
print("f1-measure: ", np.mean(NB_scores_f1))

Accuracy:  0.6431109158244931
AUC:  0.8051469067890045  0.04253460477877525
Precision:  0.56123119949292
Recall:  0.8047257731919075
Sensitivity:  0.8047257731919075
Specificity:  0.5587824398526132
f1-measure:  0.6243971854917861


Naive Bayes Splitting training and testing (70%, 30%)

In [ ]:
#Naive Bayes Splitting training and testing (70%, 30%)
NB.fit(X_train, y_train)
y_pred = NB.predict(X_test)
NB_scores2 = metrics.accuracy_score(y_test, y_pred)

tn, fp, fn, tp = matrix = confusion_matrix(y_test, y_pred).ravel()

NB_roc = roc_auc_score(y_test, y_pred)
NB_precision=tp/(tp+fp)
NB_recall=tp/(tp+fn)
NB_nvp=tn/(tn+fn)
NB_specificity=tn/(tn+fp)
NB_f1measure=2*(NB_precision*NB_recall)/(NB_precision+NB_recall)

print("Accuracy", NB_scores2)
print("ROC: ", NB_roc)
print("Precision: ", NB_precision)
print("Recall: ", NB_recall)
print("Sensitivity: ", NB_recall)
print("Specificity: ", NB_specificity)
print("F1-Measure: ", NB_f1measure)

Accuracy 0.7835820895522388
ROC:  0.7909523809523809
Precision:  0.6721311475409836
Recall:  0.82
Sensitivity:  0.82
Specificity:  0.7619047619047619
F1-Measure:  0.7387387387387387


Decision Tree 5-FOLD VALIDATION

In [ ]:
#Decision Tree 5-FOLD VALIDATION
DTree = DecisionTreeClassifier()
DTree_scores = cross_val_score(DTree, X, y, scoring = 'accuracy', cv=kf)
DTree_scores_ROC = cross_val_score(DTree, X, y, scoring = 'roc_auc', cv=kf)
DTree_scores_precision = cross_val_score(DTree, X, y, scoring = 'precision', cv=kf)
DTree_scores_recall = cross_val_score(DTree, X, y, scoring = 'recall', cv=kf)
DTree_scores_specificity = cross_val_score(DTree, X, y, scoring = specificity, cv=kf)
DTree_scores_f1 = cross_val_score(DTree, X, y, scoring = 'f1', cv=kf)

print("Accuracy", np.mean(DTree_scores))
print("AUC: ", DTree_scores_ROC.mean(), "", DTree_scores_ROC.std())
print("Precision: ", np.mean(DTree_scores_precision))
print("Recall: ", np.mean(DTree_scores_recall))
print("Sensitivity: ", np.mean(DTree_scores_recall))
print("Specificity: ", np.mean(DTree_scores_specificity))
print("f1-measure: ", np.mean(DTree_scores_f1))

Accuracy 0.7734417174063147
AUC:  0.7746282677430477  0.06432177621325617
Precision:  0.7279548003005607
Recall:  0.7035965885223835
Sensitivity:  0.7035965885223835
Specificity:  0.8194674583360225
f1-measure:  0.7098664031643633


Decision Tree Splitting training and testing (70%, 30%)

In [ ]:
#Decision Tree Splitting training and testing (70%, 30%)
DTree.fit(X_train, y_train)
y_pred = DTree.predict(X_test)
DTree_scores2 = metrics.accuracy_score(y_test, y_pred)

tn, fp, fn, tp = matrix = confusion_matrix(y_test, y_pred).ravel()

DTree_roc = roc_auc_score(y_test, y_pred)
DTree_precision=tp/(tp+fp)
DTree_recall=tp/(tp+fn)
DTree_nvp=tn/(tn+fn)
DTree_specificity=tn/(tn+fp)
DTree_f1measure=2*(DTree_precision*DTree_recall)/(DTree_precision+DTree_recall)

print("Accuracy", DTree_scores2)
print("ROC: ", DTree_roc)
print("Precision: ", DTree_precision)
print("Recall: ", DTree_recall)
print("Sensitivity: ", DTree_recall)
print("Specificity: ", DTree_specificity)
print("F1-Measure: ", DTree_f1measure)

Accuracy 0.7835820895522388
ROC:  0.760595238095238
Precision:  0.7282608695652174
Recall:  0.67
Sensitivity:  0.67
Specificity:  0.8511904761904762
F1-Measure:  0.6979166666666667


SVM 5-FOLD VALIDATION

In [ ]:
#SVM 5-FOLD VALIDATION
SVM = SVC()
SVM_scores = cross_val_score(SVM, Xzscore, y, scoring = 'accuracy', cv=kf)
SVM_scores_ROC = cross_val_score(SVM, Xzscore, y, scoring = 'roc_auc', cv=kf)
SVM_scores_ROC = cross_val_score(SVM, Xzscore, y, scoring = 'roc_auc', cv=kf)
SVM_scores_precision = cross_val_score(SVM, Xzscore, y, scoring = 'precision', cv=kf)
SVM_scores_recall = cross_val_score(SVM, Xzscore, y, scoring = 'recall', cv=kf)
SVM_scores_specificity = cross_val_score(SVM, Xzscore, y, scoring = specificity, cv=kf)
SVM_scores_f1 = cross_val_score(SVM, Xzscore, y, scoring = 'f1', cv=kf)

print("Accuracy: ", np.mean(SVM_scores))
print("AUC: ", SVM_scores_ROC.mean(), "", SVM_scores_ROC.std())
print("Precision: ", np.mean(SVM_scores_precision))
print("Recall: ", np.mean(SVM_scores_recall))
print("Sensitivity: ", np.mean(SVM_scores_recall))
print("Specificity: ", np.mean(SVM_scores_specificity))
print("f1-measure: ", np.mean(SVM_scores_f1))

Accuracy:  0.8282844768062269
AUC:  0.8694170569970623  0.02870590280826937
Precision:  0.7875557379801863
Recall:  0.7598022902403376
Sensitivity:  0.7598022902403376
Specificity:  0.8721726165022672
f1-measure:  0.7711976710287328


SVM Splitting training and testing (70%, 30%)

In [ ]:
#SVM Splitting training and testing (70%, 30%)
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
SVM_scores2 = metrics.accuracy_score(y_test, y_pred)

tn, fp, fn, tp = matrix = confusion_matrix(y_test, y_pred).ravel()

SVM_roc = roc_auc_score(y_test, y_pred)
SVM_precision=tp/(tp+fp)
SVM_recall=tp/(tp+fn)
SVM_nvp=tn/(tn+fn)
SVM_specificity=tn/(tn+fp)
SVM_f1measure=2*(SVM_precision*SVM_recall)/(SVM_precision+SVM_recall)

print("Accuracy: ", SVM_scores2)
print("ROC: ", SVM_roc)
print("Precision: ", SVM_precision)
print("Recall: ", SVM_recall)
print("Sensitivity: ", SVM_recall)
print("Specificity: ", SVM_specificity)
print("F1-Measure: ", SVM_f1measure)

Accuracy:  0.8134328358208955
ROC:  0.7965476190476192
Precision:  0.7604166666666666
Recall:  0.73
Sensitivity:  0.73
Specificity:  0.8630952380952381
F1-Measure:  0.7448979591836735
